In [2]:
import importlib
import traj_optim
import fsm_utils
importlib.reload(traj_optim)
importlib.reload(fsm_utils)
from traj_optim import TrajectoryOptimizationSolution, read_csv
from fsm_utils import (
    LEFT_STANCE, RIGHT_STANCE, SPACE_STANCE, DOUBLE_SUPPORT, PointOnFrame
)

from pydrake.all import (
    DiagramBuilder, Simulator, FindResourceOrThrow, MultibodyPlant, PiecewisePolynomial, SceneGraph,
    Parser, JointActuatorIndex, MathematicalProgram, Solve
)
from pydrake.multibody.all import JacobianWrtVariable
import pydrake.math
from pydrake.autodiffutils import AutoDiffXd
from pydrake.math import RigidTransform

import numpy as np

#builder = DiagramBuilder()
#plant = builder.AddSystem(MultibodyPlant(0.0))
file_name = "planar_walker.urdf"
#Parser(plant=plant).AddModels(file_name)

        

plant = MultibodyPlant(0.0)
parser = Parser(plant)
parser.AddModels("planar_walker.urdf")

plant.WeldFrames(
            plant.world_frame(),
            plant.GetBodyByName("base").body_frame(),
            RigidTransform.Identity()
        )

plant.Finalize()
##planar_arm = plant.ToAutoDiffXd()
plant_context = plant.CreateDefaultContext()
##context = planar_arm.CreateDefaultContext()




ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
#print(planar_arm.num_positions() , planar_arm.num_velocities())
#print(type(context))

contact_points = {
        0: PointOnFrame(
            plant.GetBodyByName("left_lower_leg").body_frame(),
            np.array([AutoDiffXd(0), AutoDiffXd(0), AutoDiffXd(-0.5)])
        ),
        1: PointOnFrame(
            plant.GetBodyByName("right_lower_leg").body_frame(),
            #np.array([AutoDiffXd(0), AutoDiffXd(0), AutoDiffXd(-0.5)])
            np.array([0, 0, -0.5])
        )
    }

plant.CalcJacobianTranslationalVelocity(
        plant_context, 
        JacobianWrtVariable.kV, 
        contact_points[1].frame, 
        contact_points[1].pt, 
        plant.world_frame(), 
        plant.world_frame()
    )

array([[ 1. ,  0. ,  1. ,  0. ,  0. , -1. , -0.5],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ]])

In [ ]:
from pydrake.autodiffutils import AutoDiffXd

prog = MathematicalProgram()

n_x = planar_arm.num_positions() + planar_arm.num_velocities()

x = np.zeros((1,  n_x ), dtype=object)
x[0] = prog.NewContinuousVariables(n_x, "x")

print(planar_arm.GetPositionsAndVelocities(context).shape)

planar_arm.SetPositionsAndVelocities(context, 
                                      x[0].reshape(-1, 1) )

(14,)


TypeError: SetPositionsAndVelocities(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.plant.MultibodyPlant_𝓣AutoDiffXd𝓤, context: pydrake.systems.framework.Context_𝓣AutoDiffXd𝓤, q_v: numpy.ndarray[object[m, 1]]) -> None
    2. (self: pydrake.multibody.plant.MultibodyPlant_𝓣AutoDiffXd𝓤, context: pydrake.systems.framework.Context_𝓣AutoDiffXd𝓤, model_instance: pydrake.multibody.tree.ModelInstanceIndex, q_v: numpy.ndarray[object[m, 1]]) -> None

Invoked with: <pydrake.multibody.plant.MultibodyPlant_𝓣AutoDiffXd𝓤 object at 0x7f5ac45263f0>, <pydrake.systems.framework.LeafContext_𝓣AutoDiffXd𝓤 object at 0x7f5ac45b0530>, array([[Variable('x(0)', Continuous)],
       [Variable('x(1)', Continuous)],
       [Variable('x(2)', Continuous)],
       [Variable('x(3)', Continuous)],
       [Variable('x(4)', Continuous)],
       [Variable('x(5)', Continuous)],
       [Variable('x(6)', Continuous)],
       [Variable('x(7)', Continuous)],
      

In [ ]:
print(planar_arm.GetPositionNames())
print(planar_arm.GetVelocityNames())

['KneedCompassGait_planar_x_x', 'KneedCompassGait_planar_z_x', 'KneedCompassGait_planar_roty_q', 'KneedCompassGait_left_hip_pin_q', 'KneedCompassGait_left_knee_pin_q', 'KneedCompassGait_right_hip_pin_q', 'KneedCompassGait_right_knee_pin_q']
['KneedCompassGait_planar_x_v', 'KneedCompassGait_planar_z_v', 'KneedCompassGait_planar_roty_w', 'KneedCompassGait_left_hip_pin_w', 'KneedCompassGait_left_knee_pin_w', 'KneedCompassGait_right_hip_pin_w', 'KneedCompassGait_right_knee_pin_w']


In [ ]:
from pydrake.solvers import MathematicalProgram, SnoptSolver
prog = MathematicalProgram()
solver = SnoptSolver()
prog.SetSolverOption(solver.id(), "Iterations limit", 2000)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ef2d9453-73df-41fc-adfb-4330c09008c0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>